In [8]:
# This is a notebook containing an attempt to implement the MLG kernel
#
# Below is a link to the reference implementation by horace:
# https://github.com/horacepan/MLGkernel
#
#
# TODO:
# 0. Write input parsing for feature nodes and test it
#
# 1. Construct graph laplacians somehow, also choose Laplacian inputs
# 2. Test LG kernel implementation
# 
# 3. Implement FLG kernel (Implement feature mapping matrices)
# 4. Test FLG kernel
# 
# 5. Implement generalized FLG kernel
# 6. Test generalized FLG kernel
#
# 7. Implement MLG kernel (May need to find a way to use MLS kernel)
# 8. Test generalized MLG kernel
#
# 9. Optimize calculations of MLG kernel
#  a. Implement finding the orthonormal basis
#  b. DP for calculation of recursion
# 10. Test final version of generalizes MLG kernel

In [9]:
# Imports 
import numpy as np

In [10]:
# Input parsing paths
adj_mat_path = "/home/mastercljohnson/MLG_KERNEL/MLGkernel/data/MUTAG.txt"
feature_path = "/home/mastercljohnson/MLG_KERNEL/MLGkernel/data/MUTAG_nodelabels.txt"

# Note: the adjacency matrix can be weighted and this code should work for the weighted case as well


In [11]:
# Input parsing

# Given the paths of the adj matrix file and feature file,
# Return both the adjacency matrices and feature vectors in a list??
def parse_files(adj_mat_p, feat_p):
    num_of_graphs = 0
    size_of_graph = 0
    
    # open the paths to the adj matrices and features
    adj_mat_f = open(adj_mat_p, 'r')
    feature_f = open(feat_p, 'r')
    
    num_of_graphs = int(adj_mat_f.readline() )
    
    # Make sure the adj matrix and feature file are for the same data
    if(num_of_graphs == int(feature_f.readline()) ):
        print("Adj Matrix and feature file have same number of graphs: ", num_of_graphs)
    else:
        print("Something is wrong with the data, num of graphs do not match for both files")
        
    adj_line = 1
    feat_line = 1
    
    # Begin parsing the files for adjacency matrices and feature files
    adj_matrix_list =[]
    feature_vec_list =[]
    
    which_graph = 0
    while (which_graph < num_of_graphs):
        read = adj_mat_f.readline()
        
        num_of_vert = int( read ) # number of vertices in this graph
        if not(num_of_vert == int(feature_f.readline()) ):
            print("Uhh Ohh, something has gone wrong with the parsing: ", num_of_vert )
        
        adj_mat = [] # construct an array of arrays for adjacency matrix
        feature_vec_vert = []
        
        for ind in range(num_of_vert):
            column = adj_mat_f.readline()
            da_real_column = np.array([int(x) for x in column.split()])
            adj_mat.append(da_real_column)
            
            label = int( feature_f.readline() )
            feature_vec_vert.append(label)
        
        # Convert type and append to giant list of adjacency matrices
        adj_mat = np.array(adj_mat)
        adj_matrix_list.append(adj_mat)
        
        feature_vec_vert = np.array(feature_vec_vert)
        feature_vec_list.append(feature_vec_vert)
        
        # Increment graph number
        which_graph +=1
        
    return adj_matrix_list, feature_vec_list
            
            
            
    
    

In [12]:
# Test the Input Parsing
a_mats, f_vecs =  parse_files(adj_mat_path,feature_path)
# print(f_vecs)

Adj Matrix and feature file have same number of graphs:  188


In [13]:
# Graph Lagrangian constructed from Adjacency matrix A
# Notice that a weighted adjacency matrix can be used for
# constructing a weighted lagrangian
def form_lap(A):
    D = np.diag(np.sum( A , axis=0))
    return D - A
    

In [14]:
# Compute Regularized Laplacian
# Given Laplacian L, eta a small constant "regularizer", I the identity matrix
def comp_reg_lap(L, eta):
    I = np.eye(L.shape[0])
    return L + eta*I

In [27]:
# LG kernel (Laplacian Graph kernel)
# Given regularized graph Laplacians L_1 and L_2 (of same size?), parameter gamma for normalization

# Optimize with log computations later on


# What is ^(-1),is it inverse?
def lg_kernel(L_1, L_2, gamma):
    # Construct identity matrices
    I1 = np.eye(L_1.shape[0])
    I2 = np.eye(L_2.shape[0])
    
    S_1 = np.linalg.inv(L_1) + gamma*I1
    S_2 = np.linalg.inv(L_2) + gamma*I2
    
    average = np.linalg.inv(S_1)/2 + np.linalg.inv(S_2)/2 
    numer = np.sqrt( abs( np.power(average, -1) ) ) # hopefully this is the 1/...
    denom = np.power( abs(S_1), 1/4 ) * np.power(abs(S_2), 1/4)
    return numer/denom

In [29]:
# Construct a graph Laplacian 
gamm = 0.001
L1 = form_lap(a_mats[0])
L2 = form_lap(a_mats[1])
reg_L1 = comp_reg_lap(L1,gamm)
reg_L2 = comp_reg_lap(L2,gamm)
# V1_feat =
# V2_feat =

# print(lg_kernel(reg_L1,reg_L1, gamm)) # has output
# print(flg_kernel(reg_L1,reg_L1, gamm)) # needs checking

In [3]:
# FLG kernel (Feature space Laplacian Graph kernel)
# Given regularized graph Laplacians L_1 and L_2
# feature mapping matrices U_1 and U_2
# parameter gamma
def flg_kernel(L_1, L_2, U_1, U_2, gamma):
    # Construct the identity matrices
    I1 = np.eye(L_1.shape[0])
    I2 = np.eye(L_2.shape[0])
    
    S_1 = U_1*np.linalg.inv(L_1)*np.transpose(U_1) + gamma*I1
    S_2 = U_2*np.linalg.inv(L_2)*np.transpose(U_2) + gamma*I2
    
    average = np.linalg.inv(S_1)/2 + np.linalg.inv(S_2)/2 
    numer = np.sqrt( abs( np.power(average, -1) ) ) # hopefully this is the 1/...
    # To check if the above is correct, compare kernel calculation with c++ impl
    denom = np.power( abs(S_1), 1/4 ) * np.power(abs(S_2), 1/4)
    return numer/denom

In [7]:
# Construction of the Gram Matrix
# Given a vertex set V_feat of features vertices with an inner product kernel of features, construct gram matrix 
def construct_gram(V_feat):
    return V_feat * V_feat

In [4]:
# Generalized FLG kernel

# Find way to construct feature matrices

# Base kernel
# Given local feature vectors v1_feat and v2_feat
def base_kernel(v1_feat,v2_feat):
    return v1_feat*v2_feat # dot product

# Given above and orthonormal basis of W = span{phi(v1),...,phi(vn1), phi(v1'),...,phi(vn2')}
def gen_flg_kernel(L_1, L_2, Q_1, Q_2 ,gamma, kern):
    # TODO: Construct Projection S_1_bar, S_2_bar
    S_1 = np.transpose(Q_1)*inv(L_1)*Q_1 + gamma*I
    S_2 = np.transpose(Q_2)*inv(L_2)*Q_2 + gamma*I
    
    average = S_1_bar^(-1)/2 + S_2_bar^(-1)/2 
    numer = sqrt( abs( average^(-1) ) )
    denom = abs(S_1_bar)^(1/4) * abs(S_2_bar)^(1/4)
    return numer/denom

In [8]:
# Construction of an orthonormal basis for the flg kernels
# Given joint Gram matrix K
def construct_Q(K):
    eigvals, r_eigvecs = np.linalg.eig(K) # compute eigenvalues and eigenvectors of K
    # Construct Q matrix for projections
    n = len(eigvals)
    Q = np.zeros((n,n))
    for j in (0,len(eigvals)):
        Q[:,j] = sqrt(eigvals[j])*r_eigvecs[:,j] # construct Q
    
    return Q

In [ ]:
# Define the neighborhoods which are nested within each layer
# IE. below the distance is how many vertices away the current vertex is (DFS)
def dis

In [9]:
# Sample vertices from a vertex set for low rank approximation of gram matrix K
# Given a vertex set V, and number of vertices to be sampled m
def sample_vset(V, m):
    return NotImplemented # some

In [6]:
# MLG kernel (Multiscale Laplacian Graph Kernel)

# Define a recursive base kernel
# Given a nested sequence of l nbhds and vertices v1,v2 
def rec_kern(v1,v2, l):
    # G_l(v1), G_l(v2)
    if (l == 1):
        return gen_flg_kernel(G_l_v1, G_2_v2, G_Q_1, G_Q_2, gamma, base_kernel)
    return gen_flg_kernel(G_l_v1, G_2_v2, G_Q_1, G_Q_2, gamma, rec_kern(l=l-1))

def mlg_kernel(L_1, L_2, Q_1, Q_2, ortho_basis ,gamma, rec_kern):
    return gen_flg_kernel(L_1, L_2, Q_1, Q_2, ortho_basis ,gamma, rec_kern)